# Maximize Sharpe Ratio Portfolio

The **Maximize Sharpe Ratio Portfolio** is a portfolio optimization strategy that aims to maximize the risk-adjusted return of a portfolio by selecting the optimal combination of assets. It is based on the concept of the Sharpe ratio, which measures the excess return per unit of risk in a portfolio.

## Key Concepts

- **Sharpe Ratio:** The Sharpe ratio is a measure of risk-adjusted return that calculates the excess return of a portfolio over the risk-free rate per unit of volatility. It is defined as:
 $$Sharpe Ratio = \frac{E(R_p) - R_f}{\sigma_p}$$
 where:
   - $E(R_p)$ is the expected return of the portfolio,
   - $R_f$ is the risk-free rate,
   - $\sigma_p$ is the standard deviation of the portfolio return.

## Characteristics

- **Objective:** To maximize the Sharpe ratio of a portfolio by selecting the optimal weights for each asset.

- **Inputs:** The inputs required for the optimization process include the expected returns of the assets, the covariance matrix of asset returns, and the risk-free rate.

- **Optimization:** The optimization process involves finding the weights that maximize the Sharpe ratio of the portfolio, subject to the constraint that the sum of the weights equals one.

- **Output:** The output of the optimization process is the optimal weights for each asset in the portfolio that maximize the Sharpe ratio.

## Limitations

- **Assumptions:** The Maximize Sharpe Ratio Portfolio assumes that asset returns are normally distributed and that historical data is an accurate representation of future returns.

- **Sensitivity:** The optimization process is sensitive to the inputs, such as expected returns and the covariance matrix, which can impact the results.

- **Risk-Free Rate:** The choice of the risk-free rate can also affect the optimization results, as it is used in the calculation of the Sharpe ratio.

## Usage example

An usage example of strategy based on the markowitz min variance portfolio.

Your task is to propose a particular strategy similar to the one located at the folder `strategy`. 

In [1]:
import sys
!{sys.executable} -m pip install -r requirements.txt --quiet

In [10]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import quantstats as qs

## Load Data

You may organize the data the way you want. Here, we downloaded all data into the `dataset` directory in parquet format.

We also use auxiliar functions from the `data_market` directory to handle these data.

In [ ]:
from data_market.datalake import load_data

dict_data = load_data()

# Let's check the keys of the dictionary, each one a DataFrame
print(dict_data.keys())

In [ ]:
# Let's check the first DataFrame: prices of US stocks
dict_data['stocks']

## Strategy execution

Following, we test the execution of our strategy: for just a single day.

In [12]:
# You must write all the code for your strategy entirely in the strategy directory
# Your strategy must implement the StrategyInterface interface defined in simulator/strategy_interface.py
from strategy.strategy_max_sharpe import MaxSharpeStrategy

strategy = MaxSharpeStrategy()

In [ ]:
# Execution for a single day
weights = strategy.calculate_next_weights(data = dict_data, t = 500)

# Check if the returned DataFrame has the correct format
assert strategy.check_return(weights)

Your strategy must return a DataFrame with the columns: `date`, `ticker` and `weights`.

In [ ]:
weights

## Simulation

Following we simulate our strategy and generate the final report.

In [ ]:
from simulator.strategy_simulator import strategy_simulator

# Initialize data structures to store results
ret_port = pd.Series(dtype="float64")  # Series to store portfolio returns
weights_db = pd.DataFrame()  # DataFrame to store historical weights

# Loop through a range of time values
for t in range(500, len(dict_data['stocks'].index) - 1):

    # Use the strategy simulator to get portfolio's historical weights [weights_db]
    # and its next day returns [ret_port]
    ret_port, weights_db = strategy_simulator(path = "results/",
                                              strategy = strategy,
                                              data = dict_data,
                                              t = t,
                                              ret_port = ret_port,
                                              weights_db = weights_db)

## Report

We can use the results of the simulation, saved in the directory `results`, to generate a report of our strategy using `quantstats`.

The simulation generate two parquet files:

- [ret_port.parquet](results/ret_port.parquet): DataFrame with the return of the portfolio, for each simulated datetime
- [wegiths_db.parquet](results/weights_db.parquet): DataFrame with the weights of each stock in the portfolio, for each simulated datetime

In [ ]:
ret_port = pd.read_parquet("results/ret_port.parquet")
ret_port['date'] = pd.to_datetime(ret_port['date'], format = "%Y-%m-%d")
ret_port = ret_port.set_index("date").ret_port

In [ ]:
ret_port.head(3)

Following we generate a HTML report, comparing our strategy with the SP500.

In [ ]:
qs.reports.html(ret_port, "SPY", text_description="""
    <p>Demonstration of a simple strategy.</p>
    <p><strong>Important</strong>: Transaction costs are not taken into account.</p>""")